# Media Mounter

> Mounts media directories as static files for serving through the web server

In [ ]:
#| default_exp media.mounter

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import hashlib
from pathlib import Path
from typing import Dict, Optional, List
from fastcore.basics import patch

from starlette.staticfiles import StaticFiles

## MediaMounter Class

Handles mounting media directories as static file routes in the FastHTML application. Each directory gets a unique URL prefix based on a hash of its path.

In [ ]:
#| export
class MediaMounter:
    """Mounts directories for static file serving with instance-level state.

    Each workflow instance gets its own mounter with its own registry of
    mounted directories, ensuring proper isolation between workflow instances.

    The mounter uses a unique prefix pattern (sf_media_{hash}) to avoid
    conflicts with other mounters or static file routes.
    """

    # Prefix pattern for this mounter's routes
    ROUTE_PREFIX_PATTERN = "sf_media_"

    def __init__(self):
        """Initialize the mounter with empty state."""
        self._mounted: Dict[str, str] = {}  # directory -> route_prefix
        self._app = None

In [ ]:
#| export
@patch
def mount(self: MediaMounter,
          app,  # FastHTML app instance
          directories: List[str]  # Directory to mount
         ) -> str:  # URL prefix for the mounted directory
    """Mount directories to app for static file serving.

    This method clears any existing mounts from this instance before
    adding new ones, ensuring a clean state on each call.

    Args:
        app: FastHTML/Starlette application instance.
        directories: List of directory paths to mount.
    """
    self._app = app
    self._mounted.clear()

    # Remove any existing mounts from this instance
    self._remove_existing_mounts(app)

    for directory in directories:
        self._mount_directory(app, directory)

In [ ]:
#| export
@patch
def get_url(self: MediaMounter,
            file_path: str  # Full path to the media file
           ) -> Optional[str]:  # URL to access the file or None
    """Get URL for a file based on mounted directories.

    Args:
        file_path: Full path to the media file.

    Returns:
        URL path to access the file, or None if not in a mounted directory.
    """
    file_path_resolved = Path(file_path).resolve()

    for directory, prefix in self._mounted.items():
        dir_path = Path(directory).resolve()
        try:
            relative_path = file_path_resolved.relative_to(dir_path)
            return f"/{prefix}/{relative_path.as_posix()}"
        except ValueError:
            # File is not in this directory
            continue

    return None

In [ ]:
#| export
@patch
def is_mounted(self: MediaMounter,
               directory: str  # Directory to check
              ) -> bool:  # True if directory is mounted
    """Check if a directory is currently mounted.

    Args:
        directory: Directory path to check.

    Returns:
        True if the directory is mounted.
    """
    return directory in self._mounted

In [ ]:
#| export
@patch
def get_mounted_directories(self: MediaMounter) -> List[str]:
    """Get list of currently mounted directories.

    Returns:
        List of mounted directory paths.
    """
    return list(self._mounted.keys())

In [ ]:
#| export
@patch
def unmount_all(self: MediaMounter) -> None:
    """Remove all mounts from this instance."""
    if self._app:
        self._remove_existing_mounts(self._app)
    self._mounted.clear()

In [ ]:
#| export
@patch
def _mount_directory(self: MediaMounter, 
                     app, 
                     directory: str) -> None:
    """Mount a single directory.

    Args:
        app: FastHTML/Starlette application instance.
        directory: Directory path to mount.
    """
    dir_path = Path(directory)

    if not dir_path.exists():
        print(f"[MediaMounter] Warning: Directory does not exist: {directory}")
        return

    if not dir_path.is_dir():
        print(f"[MediaMounter] Warning: Path is not a directory: {directory}")
        return

    prefix = self._generate_prefix(directory)

    try:
        mount = Mount(
            f"/{prefix}",
            app=StaticFiles(directory=str(dir_path)),
            name=prefix
        )

        # Insert at the beginning of routes (before other routes)
        app.routes.insert(0, mount)

        # Store the mapping
        self._mounted[directory] = prefix

    except Exception as e:
        print(f"[MediaMounter] Error mounting {directory}: {e}")

In [ ]:
#| export
@patch
def _generate_prefix(self: MediaMounter, 
                     directory: str) -> str:
    """Generate a unique route prefix for a directory.

    Uses MD5 hash of the directory path to ensure uniqueness while
    keeping the prefix reasonably short.

    Args:
        directory: Directory path.

    Returns:
        Route prefix string (e.g., "sf_media_abc12345").
    """
    dir_hash = hashlib.md5(directory.encode()).hexdigest()[:8]
    return f"{self.ROUTE_PREFIX_PATTERN}{dir_hash}"

In [ ]:
#| export
@patch
def _remove_existing_mounts(self: MediaMounter, 
                            app) -> None:
    """Remove existing mounts from this instance.

    Only removes mounts that match this mounter's prefix pattern.

    Args:
        app: FastHTML/Starlette application instance.
    """
    # Filter out mounts with our prefix pattern
    app.routes[:] = [
        route for route in app.routes
        if not (
            isinstance(route, Mount) and
            route.path.startswith(f"/{self.ROUTE_PREFIX_PATTERN}")
        )
    ]

## Usage Examples

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()